# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [ ]:
#!pip install python-dotenv
#!pip install openai

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [65]:
useLogging = False # set to True to get logging information (and hopefully track which LLM is called when)
use_Ollama_For_Inference = True # set to True to use Ollama inference models (and pull at least the gemma:2b model)

import openai
#Defaults to OpenAI if use_Ollama_For_Inference=False and use_Ollama_For_Inference=False
openai.api_base = inferApiBase = embedApiBase =  "https://api.openai.com/v1"
openai.base_url = inferBaseUrl = embedBaseUrl = "https://api.openai.com"
openai.api_key = inferApiKey = embedApiKey = os.environ['OPENAI_API_KEY']
infer_model_name = "gpt-3.5-turbo"
llm_platform = "openai"


if use_Ollama_For_Inference:
    inferApiBase = "http://localhost:11434/v1"
    inferBaseUrl = "http://localhost:1143"
    inferApiKey = "ollama"
    infer_model_name = "llama3:8b" #you can/should customize this to test different Ollama LLMs. Use the NAME field from `ollama list`

print('Infer API Key:', inferApiKey)
print('Inference Model:', infer_model_name)

import langchain
if useLogging:
    import logging
    import requests 


    logging.basicConfig(level=logging.DEBUG,
                        format='%(asctime)s - %(levelname)s - %(filename)s:%(funcName)s:%(lineno)d - %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')
    requests.packages.urllib3.add_stderr_logger()
    OLLAMA_DEBUG=1
    langchain.debug = True
else:
    OLLAMA_DEBUG=0
    langchain.debug = False    

Infer API Key: ollama
Inference Model: llama3:8b


## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message.content





In [3]:
get_completion("What is 1+1?")

'1+1 equals 2.'

In [4]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [5]:
style = """American English \
in a calm and respectful tone
"""

In [6]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [7]:
response = get_completion(prompt)

In [8]:
response

"I am really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I need your help right now, friend!"

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [ ]:
#!pip install --upgrade langchain

### Model

In [4]:
from langchain_openai import ChatOpenAI

In [67]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, base_url=inferApiBase, model=infer_model_name)
chat

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000023164FA1C00>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000023164FA3310>, model_name='llama3:8b', temperature=0.0, openai_api_key=SecretStr('**********'), openai_api_base='http://localhost:11434/v1', openai_proxy='')

### Prompt template

In [69]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
Only output the translation, \
without any rephrasing of the translation request.  \
text: ```{text}```
"""

In [70]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [24]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. Only output the translated text,  without any rephrasing or repetition of the translation request.  text: ```{text}```\n')

In [25]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [26]:
customer_style = """American English \
in a calm and respectful tone
"""

In [27]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [71]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [14]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [72]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. Only output the translation, without any rephrasing of the translation request.  text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


In [74]:
# Call the LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages)

In [75]:
print(customer_response.content)

I'm absolutely furious that my blender lid came loose and covered my kitchen walls with a mess! To make things even more frustrating, the warranty doesn't cover the cost of cleaning up this disaster. I really need your assistance right away!


In [76]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [77]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [78]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. Only output the translation, without any rephrasing of the translation request.  text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [79]:
service_response = chat.invoke(service_messages)
print(service_response.content)

Arrrr, greetings me hearty customer! Yer warranty don't cover the cost o' cleanin' yer kitchen, matey, 'cause ye be the one who got yerself into this pickle by forgettin' to put the lid on yer blender afore startin' it up. Shiver me timbers! Ye got yerself mixed up in a bit o' trouble, don't ye? Well, I reckon that's just the way the cookie crumbles, matey. So, I'll be seein' ye off, savvy?


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [80]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [81]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [82]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [83]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0, base_url=inferApiBase, model=infer_model_name)
response = chat.invoke(messages)
print(response.content)


Here is the extracted information in JSON format:

```
{
    "gift": True,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there", "but I think it's worth it for the extra features."]
}
```


In [84]:
type(response.content)

str

In [48]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a Python dictionary

In [85]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [87]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [108]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [100]:
format_instructions = output_parser.get_format_instructions()


In [104]:
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.\n\t"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.\n\t"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.\n}\n```'

In [91]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [117]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [118]:
response = chat.invoke(messages)

In [94]:
print(response.content)

Here is the extracted information in the requested format:

```json
{
    "gift": True,
    "delivery_days": "2",
    "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```

Note that the `price_value` field contains a list of strings because the text mentions the price in multiple sentences.


In [144]:
#we need to parse the previous output since it's not purely JSON code

json_format_template = """\
For the following text, extract only the information inside the leading and trailing "```json" and "```".
Replace any True value as "1" and any False value as "0". 
Exclude any introductory phrase from your response. 

text: {text}

"""

json_prompt = ChatPromptTemplate.from_template(template=json_format_template)

json_messages = json_prompt.format_messages(text=response.content)

json_response = chat.invoke(json_messages)
print(json_response.content)

{
    "gift": 1,
    "delivery_days": "2",
    "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


In [140]:
output_dict = output_parser.parse(json_response.content)

In [141]:
output_dict

{'gift': 1,
 'delivery_days': '2',
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [133]:
type(output_dict)

dict

In [142]:
output_dict.get('delivery_days')

'2'